# Packages

In [ ]:
import numpy as np
from numpy.typing import NDArray
from pathlib import Path
from scipy.spatial.distance import pdist, squareform

# Read data

In [ ]:
def load_tsp(path: Path, header: int = 6, footer: int = 1) -> NDArray:
    """load_model(path)

    Load a tsp instance from a path and store it into a NDArray.
    A user can download the instances from
    http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/

    Parameters
    ----------
    path : Path
        Path or location of the stan model.
    header : int, optional
        Lines to skip from header, by default 6.
    footer : int, optional
        Lines to skip from footer, by default 1.

    Returns
    -------
    NDArray
        Matrix of distances between each pair of entries.
    """
    coords = np.genfromtxt(path, skip_header=header, skip_footer=footer)[:, 1:]
    dist_array = pdist(coords)
    return squareform(dist_array)

# Base Functions

In [ ]:
def random_solution(nodes: list | NDArray, seed: int | None = None) -> NDArray:
    """random_solution(nodes)

    Function that returns an random solution for a TSP,
    based on a random permutation of a list or an array
    named `nodes`.

    Parameters
    ----------
    nodes : list | NDArray
        List or array of nodes to be visited in a TSP.
    seed : int | None, optional
        Seed for pseudo-random numbers generation, by default None.

    Returns
    -------
    NDArray
        Array of a permutation with the same elements as `nodes`.

    Raises
    ------
    TypeError
        `nodes` must be a 1D list or 1D array.
    """
    if np.ndim(nodes) != 1:
        raise TypeError(f"nodes = {nodes} is not a 1D list or a 1D NDArray")
    else:
        return np.random.RandomState(seed).permutation(nodes)

def mp_mutation(chromosome: NDArray, seed: int | None = None) -> NDArray:
    """mp_mutation(chromosome)

    Multi-point inversion mutation. A random mask encodes
    which elements will keep the original order or the
    reversed one.

    Parameters
    ----------
    chromosome : NDArray
        Encoding of a solution (chromosome).
    seed : int | None, optional
        Seed for pseudo-random numbers generation, by default None.

    Returns
    -------
    NDArray
        Returns the chromosome after mutation.
    """
    individual = chromosome.copy()
    mask = np.random.RandomState(seed).randint(2, size=len(individual)) == 1
    individual[~mask] = np.flip(individual[~mask])
    return individual

def mp_crossover(chromosome_a: NDArray,
              chromosome_b: NDArray,
              seed: int | None = None) -> tuple[NDArray, NDArray]:
    """mp_crossover(chromosome_a, chromosome_b)

    Multi-point ordered crossover.

    Parameters
    ----------
    chromosome_a : NDArray
        Encoding of a solution (chromosome).
    chromosome_b : NDArray
        Encoding of a solution (chromosome).
    seed : int | None, optional
        Seed for pseudo-random numbers generation, by default None.

    Returns
    -------
    tuple[NDArray, NDArray]
        Tuple of chromosomes after crossover.
    """
    child_a = chromosome_a.copy()
    child_b = chromosome_b.copy()
    mask = np.random.RandomState(seed).randint(2, size=len(chromosome_a)) == 1
    child_a[~mask] = sorted(child_a[~mask], key=lambda x: np.where(chromosome_b == x))
    child_b[mask] = sorted(child_b[mask], key=lambda x: np.where(chromosome_a == x))
    return child_a, child_b

# GA Functions

In [ ]:
def initial_population(nodes: list | NDArray, size: int, seed: int | None = None) -> list[NDArray]:
    """initial_population(nodes, size)

    Function that returns an initial population for a TSP,
    based on a random permutations of a list or an array
    named `nodes`.

    Parameters
    ----------
    nodes : list | NDArray
        List or array of nodes to be visited in a TSP.
    size : int
        Number of chromosomes of the population.
    seed : int | None, optional
        Seed for pseudo-random numbers generation, by default None.

    Returns
    -------
    list[NDArray]
        Population or list of solutions.

    Raises
    ------
    TypeError
        `nodes` must be a 1D list or 1D array.
    ValueError
        The population must have at least one member.
    """
    if np.ndim(nodes) != 1:
        raise TypeError(f"nodes = {nodes} is not a 1D list or a 1D NDArray")
    elif size < 1:
        raise ValueError(f"size = {size} must be equal or greater than one.")
    elif seed:
        return [random_solution(nodes, seed + i) for i in range(size)]
    else: return [random_solution(nodes, seed) for _ in range(size)]

def selection(population: list[NDArray],
              values: NDArray,
              tournament_size: int = 3,
              seed: int | None = None) -> NDArray:
    """selection(population, values)

    Perform tournament selection in the `population`. The selection
    process rely on randomly choosing a number chromosomes defined by the
    parameter `tournament_size`. After, the one with the minimum value is
    extracted from the selected group.

    Parameters
    ----------
    population : list[NDArray]
        List of chromosomes.
    values : NDArray
        List of chromosomes' values.
    tournament_size : int, optional
        Number of chromosomes randomly selected
        for the tournament, by default 3.
    seed : int | None, optional
        Seed for pseudo-random numbers generation, by default None.

    Returns
    -------
    NDArray
        The tournament's winner chromosome.
    """

    idx = np.random.RandomState(seed).choice(range(len(population)), size=tournament_size, replace=False)
    winner = np.argmin(values[idx])
    return population[winner]

def mutation(chromosome: NDArray,
             mutation_rate: float = 0.15,
             seed: int | None = None) -> NDArray:
    """mutation(chromosome)

    Perform slight changes in a chromosome.

    Parameters
    ----------
    chromosome : NDArray
        Encoding of a solution (chromosome).
    mutation_rate : float, optional
        probability, between 0 and 1, of happening
        a mutation, by default 0.15.
    seed : int | None, optional
        Seed for pseudo-random numbers generation, by default None.

    Returns
    -------
    NDArray
        Returns the chromosome after mutation.
    """
    if mutation_rate > np.random.RandomState(seed).uniform(0, 1):
        return mp_mutation(chromosome, seed)
    else:
        return chromosome

def crossover(chromosome_a: NDArray,
              chromosome_b: NDArray,
              crossover_rate: float = 0.85,
              seed: int | None = None) -> tuple[NDArray, NDArray]:
    """crossover(chromosome_a, chromosome_b)

    Combine attributes from both chromosomes.

    Parameters
    ----------
    chromosome_a : NDArray
        Encoding of a solution (chromosome).
    chromosome_b : NDArray
        Encoding of a solution (chromosome).
    crossover_rate : float, optional
        probability, between 0 and 1, of happening
        a crossover, by default 0.85.
    seed : int | None, optional
        Seed for pseudo-random numbers generation, by default None.

    Returns
    -------
    tuple[NDArray, NDArray]
        Tuple of chromosomes after crossover.
    """
    if crossover_rate > np.random.RandomState(seed).uniform(0, 1):
        return mp_crossover(chromosome_a, chromosome_b, seed)
    else:
        return chromosome_a, chromosome_b

def update(population: list[NDArray],
           values: NDArray,
           crossover_rate: float = 0.85,
           mutation_rate: float = 0.15,
           tournament_size: int = 3,
           seed: int | None = None) -> list[NDArray]:
    """update(population, values)

    Combine the tournament selection, crossover and mutation for the
    entire population, generating a new one.

    Parameters
    ----------
    population : list[NDArray]
        List of chromosomes.
    values : NDArray
        List of chromosomes' values.
    crossover_rate : float, optional
        probability, between 0 and 1, of happening
        a crossover, by default 0.85.
    mutation_rate : float, optional
        probability, between 0 and 1, of happening
        a mutation, by default 0.15.
    tournament_size : int, optional
        Number of chromosomes randomly selected
        for the tournament, by default 3.
    seed : int | None, optional
        Seed for pseudo-random numbers generation, by default None.

    Returns
    -------
    list[NDArray]
        New population or list of solutions.
    """
    size = len(population)
    selected = [selection(population, values, tournament_size, seed) for _ in range(size)]
    offspring = []
    for parents in zip(selected[0::2], selected[1::2]):
        chromosome_a, chromosome_b = parents
        child_a, child_b = crossover(chromosome_a, chromosome_b, crossover_rate, seed)
        child_a = mutation(child_a, mutation_rate, seed)
        child_b = mutation(child_b, mutation_rate, seed)
        offspring.append(child_a)
        offspring.append(child_b)
    return offspring

def evaluate(chromosome: NDArray, distances: NDArray):
    back_to_origin = distances[chromosome[-1], 0]
    return back_to_origin + sum(distances[pair] for pair in zip(chromosome, chromosome[1:]))


# Run

In [ ]:
iterations = range(1, 501)
log = []

In [ ]:
distances = load_tsp(Path("./a280.tsp"))
nodes = list(range(len(distances)))

In [ ]:
pop = initial_population(nodes, 100)
values = np.array(list(map(lambda individual: evaluate(individual, distances), pop)))
best_idx = np.argmin(values)
log.append({"tour": pop[best_idx], "value": values[best_idx]})

In [ ]:
for i in iterations:
    pop = update(pop, values)
    values = np.array(list(map(lambda individual: evaluate(individual, distances), pop)))
    best_idx = np.argmin(values)
    if values[best_idx] < log[-1]["value"]:
        log.append({"tour": pop[best_idx], "value": values[best_idx]})
    else:
        log.append(log[-1])